# Notebook pro trénink s destilací nad datasetem CIFAR100
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR100, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [1]:
from transformers import Trainer, EarlyStoppingCallback, AutoModelForImageClassification
from torch.utils.data import ConcatDataset, DataLoader
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()
DATASET = "cifar100"

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 1g.10gb


Provedení transformací nad datasetem.

In [4]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform)
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform)


In [5]:
augment_transform = base.aug_transforms()

train_aug = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)

In [6]:
train_part_cpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
train_part_gpu = base.CustomCIFAR100(root=f"{os.path.expanduser('~')}/data/100", train=True, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)

In [7]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
print(len(train_aug))
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

25912


### Standardní trénink náhodně inicializovaného modelu. 

In [85]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-basetrain", logging_dir=f"~/logs/{DATASET}/vit-basetrain", lr=0.0002, warmup_steps=25, epochs=20)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [87]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.889100,0.907664,0.747100,0.767294,0.747100,0.746593
2,0.595300,0.687753,0.799200,0.809146,0.799200,0.797973
3,0.365300,0.603011,0.823700,0.833367,0.823700,0.824719
4,0.232100,0.649604,0.813800,0.824856,0.813800,0.814173
5,0.158200,0.631279,0.827700,0.835012,0.827700,0.827885
6,0.108600,0.691468,0.821500,0.826127,0.821500,0.820630
7,0.079900,0.693256,0.830000,0.837371,0.830000,0.830322
8,0.058700,0.711615,0.831200,0.835736,0.831200,0.831258
9,0.042100,0.742160,0.832300,0.836948,0.832300,0.832270
10,0.033400,0.764705,0.832600,0.839053,0.832600,0.833081


TrainOutput(global_step=6260, training_loss=0.1815786790662109, metrics={'train_runtime': 2853.1255, 'train_samples_per_second': 280.394, 'train_steps_per_second': 2.194, 'total_flos': 3.6877173129216e+18, 'train_loss': 0.1815786790662109, 'epoch': 20.0})

In [88]:
model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [89]:
trainer.evaluate(test)

{'eval_loss': 0.8002419471740723,
 'eval_accuracy': 0.8539,
 'eval_precision': 0.855637204074962,
 'eval_recall': 0.8539,
 'eval_f1': 0.8540311959806233,
 'eval_runtime': 14.5973,
 'eval_samples_per_second': 685.057,
 'eval_steps_per_second': 5.412,
 'epoch': 20.0}

In [13]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-basetrain.pth")

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [17]:
base.reset_seed()

In [18]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-distilltrain", logging_dir=f"~/logs/{DATASET}/vit-distilltrain", remove_unused_columns=False, epochs=20, lr=0.00026, warmup_steps=25, weight_decay=0.002, lambda_param=.8, temp=4)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.249400,1.135772,0.743700,0.766450,0.743700,0.744170
2,0.793600,0.902559,0.782500,0.797615,0.782500,0.782237
3,0.544500,0.784060,0.804200,0.815616,0.804200,0.804493
4,0.401300,0.721304,0.816300,0.827228,0.816300,0.817295
5,0.325400,0.704685,0.821400,0.831154,0.821400,0.822258
6,0.273100,0.698496,0.824600,0.833959,0.824600,0.824671
7,0.235400,0.661701,0.835900,0.843434,0.835900,0.836983
8,0.205400,0.646265,0.832500,0.840137,0.832500,0.833397
9,0.186800,0.613126,0.841800,0.847736,0.841800,0.842513
10,0.170800,0.614429,0.838600,0.845444,0.838600,0.839597


TrainOutput(global_step=6260, training_loss=0.3249329813753073, metrics={'train_runtime': 2851.7587, 'train_samples_per_second': 280.529, 'train_steps_per_second': 2.195, 'total_flos': 3.6877173129216e+18, 'train_loss': 0.3249329813753073, 'epoch': 20.0})

In [21]:
student_model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [22]:
trainer.evaluate(test)

{'eval_loss': 0.4296223223209381,
 'eval_accuracy': 0.857,
 'eval_precision': 0.8619413810627935,
 'eval_recall': 0.8570000000000003,
 'eval_f1': 0.8575483875362276,
 'eval_runtime': 14.3963,
 'eval_samples_per_second': 694.622,
 'eval_steps_per_second': 5.488,
 'epoch': 20.0}

In [23]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-distilltrain.pth")

In [47]:
base.reset_seed()

In [48]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-base-aug", logging_dir=f"~/logs/{DATASET}/vit-base-aug", lr=0.0002, weight_decay=0.006,	 warmup_steps=25, epochs=20)
model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.479400,0.804270,0.767400,0.783080,0.767400,0.765975
2,0.469600,0.647720,0.807600,0.817776,0.807600,0.806484
3,0.274900,0.682390,0.811700,0.822020,0.811700,0.810421
4,0.178400,0.642784,0.827000,0.832708,0.827000,0.826682
5,0.121800,0.730990,0.816400,0.824934,0.816400,0.816781
6,0.089500,0.734367,0.820700,0.830008,0.820700,0.819980
7,0.066300,0.755954,0.826300,0.833348,0.826300,0.827101
8,0.050700,0.755929,0.834200,0.837450,0.834200,0.833732
9,0.039600,0.796329,0.832000,0.836739,0.832000,0.831415
10,0.028100,0.820643,0.835600,0.839777,0.835600,0.835426


TrainOutput(global_step=10300, training_loss=0.1429132774942419, metrics={'train_runtime': 4491.7074, 'train_samples_per_second': 293.483, 'train_steps_per_second': 2.293, 'total_flos': 6.076620588232212e+18, 'train_loss': 0.1429132774942419, 'epoch': 20.0})

In [51]:
model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [52]:
trainer.evaluate(test)

{'eval_loss': 0.8661243319511414,
 'eval_accuracy': 0.854,
 'eval_precision': 0.8560095977657519,
 'eval_recall': 0.8539999999999999,
 'eval_f1': 0.8541807597168135,
 'eval_runtime': 14.5049,
 'eval_samples_per_second': 689.421,
 'eval_steps_per_second': 5.446,
 'epoch': 20.0}

In [53]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-basetrain-aug.pth")

Destilace

In [79]:
base.reset_seed()

In [80]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/vit-distill-aug", logging_dir=f"~/logs/{DATASET}/vit-distill-aug", remove_unused_columns=False, weight_decay=0.007, epochs=20, lr=0.00023, lambda_param=.8, temp=4)
student_model = AutoModelForImageClassification.from_pretrained("timm/tiny_vit_5m_224.in1k", num_labels=100, ignore_mismatched_sizes=True)

Some weights of TimmWrapperForImageClassification were not initialized from the model checkpoint at timm/tiny_vit_5m_224.in1k and are newly initialized because the shapes did not match:
- head.fc.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([100]) in the model instantiated
- head.fc.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([100, 320]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_combo,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [82]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.740700,0.950765,0.777300,0.785016,0.777300,0.776550
2,0.670400,0.788590,0.806100,0.816434,0.806100,0.806857
3,0.469600,0.729935,0.820700,0.826967,0.820700,0.820080
4,0.366900,0.682700,0.827300,0.835293,0.827300,0.827602
5,0.305900,0.679056,0.830200,0.837416,0.830200,0.830512
6,0.263600,0.638735,0.836400,0.842191,0.836400,0.837108
7,0.235800,0.651164,0.835300,0.842438,0.835300,0.836101
8,0.213100,0.621165,0.839100,0.845348,0.839100,0.839817
9,0.193900,0.605474,0.844100,0.848942,0.844100,0.844689
10,0.178600,0.583228,0.846200,0.851931,0.846200,0.847011


TrainOutput(global_step=9785, training_loss=0.303062785542858, metrics={'train_runtime': 4270.5001, 'train_samples_per_second': 308.685, 'train_steps_per_second': 2.412, 'total_flos': 5.772789558820602e+18, 'train_loss': 0.303062785542858, 'epoch': 19.0})

In [83]:
student_model.eval()

TimmWrapperForImageClassification(
  (timm_model): TinyVit(
    (patch_embed): PatchEmbed(
      (conv1): ConvNorm(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): GELU(approximate='none')
      (conv2): ConvNorm(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stages): Sequential(
      (0): ConvLayer(
        (blocks): Sequential(
          (0): MBConv(
            (conv1): ConvNorm(
              (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): ConvNorm(
              (c

In [84]:
trainer.evaluate(test)

{'eval_loss': 0.45350322127342224,
 'eval_accuracy': 0.854,
 'eval_precision': 0.8582488555626767,
 'eval_recall': 0.8540000000000001,
 'eval_f1': 0.8544667534154797,
 'eval_runtime': 14.7511,
 'eval_samples_per_second': 677.916,
 'eval_steps_per_second': 5.356,
 'epoch': 19.0}

In [43]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/vit-distilltrain_aug.pth")

In [44]:
base.count_parameters(student_model)

model size: 21.339MB.
Total Trainable Params: 5103864.


,Modules,Parameters
0,timm_model.patch_embed.conv1.conv.weight,864
1,timm_model.patch_embed.conv1.bn.weight,32
2,timm_model.patch_embed.conv1.bn.bias,32
3,timm_model.patch_embed.conv2.conv.weight,18432
4,timm_model.patch_embed.conv2.bn.weight,64
...,...,...
210,timm_model.stages.3.blocks.1.local_conv.bn.bias,320
211,timm_model.head.norm.weight,320
212,timm_model.head.norm.bias,320
213,timm_model.head.fc.weight,32000


In [45]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  38.48 ms
  1 measurement, 1000 runs , 6 threads


In [46]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  10.04 ms
  1 measurement, 1000 runs , 6 threads
